In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes
import py_vollib_vectorized

grid definition

In [ ]:
texp_dim=np.linspace(1,0,10)
spot_dim=np.arange(.05,2,.05)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

option definition

In [74]:
flag = 'c'  # 'c' for call, 'p' for put
K = 1  # Strikes
r=0
iv = 0.2  # Implied Volatilities
option_price=py_vollib.black_scholes.black_scholes(flag, spot_grid, K, texp_grid, r, iv,return_as='array')
option_greeks = py_vollib_vectorized.get_all_greeks(flag, spot_grid, K, texp_grid, r, iv, return_as='dict')  


In [75]:
len(option_greeks["delta"])
gr_names=list(option_greeks.keys())
option_greeks_arr={}
for gr in gr_names:
    arr=np.array(option_greeks[gr])
    arr=arr.reshape(len(texp_dim),len(spot_dim))
    option_greeks_arr[gr]=arr
option_greeks_arr["price"]=option_price.reshape(len(texp_dim),len(spot_dim))
option_greeks_arr["price"].shape
spot_grid.shape



(20, 39)

In [76]:
df = pd.DataFrame(option_greeks_arr["price"], columns = spot_dim, index = texp_dim)
df.iloc[:,19]

1.00    0.079656
0.95    0.077645
0.90    0.075581
0.85    0.073457
0.80    0.071270
0.75    0.069013
0.70    0.066678
0.65    0.064258
0.60    0.061742
0.55    0.059119
0.50    0.056372
0.45    0.053484
0.40    0.050429
0.35    0.047176
0.30    0.043680
0.25    0.039878
0.20    0.035671
0.15    0.030894
0.10    0.025227
0.05    0.017840
Name: 1.0, dtype: float64


cp=["p"]

In [94]:
#90/110 risk reversal
cp =['p','c']
texp=[1,2]
r= 0
v = [0.2,0.2]  # Implied Volatilities
K = [0.9,1.1]  # Strikes
qty=[-1,1]

In [157]:
num_time_steps = 10
num_spots= 21
low_strike=.5   
high_strike=2

prepare for pricing

In [165]:
max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
spot_dim=np.linspace(low_strike,high_strike,num_spots)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

all_option_greeks={
    "delta":[],
    "gamma":[],
    "theta":[],
    "vega":[],
    "rho":[],
    "price":[]
}

for (qtyi,texpi,ki,cpi, ivi) in zip(qty,texp,K,cp,v):
    texp_gridi=texp_grid-(max_texp-texpi)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    option_price=py_vollib.black_scholes.black_scholes(cpi, spot_grid, ki, texp_gridi_zero, r, ivi,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(cpi, spot_grid, ki, texp_gridi_zero, r, ivi, return_as='dict')
    gr_names=list(option_greeks.keys())
    option_greeks_arr={}
    for gr in gr_names:
        arr=np.array(option_greeks[gr])
        arr=arr.reshape(len(texp_dim),len(spot_dim))
        option_greeks_arr[gr]=arr
        all_option_greeks[gr].append(arr)
    arr=np.array(option_price).reshape(len(texp_dim),len(spot_dim))
    #arr=option_price.reshape(len(texp_dim),len(spot_dim))
    all_option_greeks["price"].append(arr)

portfolio_greeks={}
for gr in list(all_option_greeks.keys()):
    for (qtyi,i) in zip(qty,range(len(qty))):
        if portfolio_greeks.get(gr) is None:
            portfolio_greeks[gr]=qtyi*all_option_greeks[gr][i]
        else:
            portfolio_greeks[gr]=portfolio_greeks[gr]+qtyi*all_option_greeks[gr][i]


#for (gr, qtyi) in zip(list(all_option_greeks.keys()),qty):
#    portfolio_greeks[gr]=np.sum(np.array(all_option_greeks[gr]*qtyi),axis=0)


In [167]:
a=np.sum(np.array(all_option_greeks["rho"]),axis=0)
all_option_greeks["price"][0].shape
#portfolio_greeks["price"]

(10, 21)

In [168]:
#all_option_greeks.keys()
#portfolio_greeks["price"].shape
df = pd.DataFrame(portfolio_greeks["gamma"], columns = spot_dim, index = texp_dim)

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler


df.style.format('{:.2f}').format_index('{:.2f}').background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2)

,0.500000,0.575000,0.650000,0.725000,0.800000,0.875000,0.950000,1.025000,1.100000,1.175000,1.250000,1.325000,1.400000,1.475000,1.550000,1.625000,1.700000,1.775000,1.850000,1.925000,2.000000
2.00,0.01,-0.11,-0.46,-0.90,-1.13,-0.99,-0.58,-0.10,0.28,0.51,0.60,0.58,0.52,0.43,0.34,0.26,0.20,0.15,0.11,0.08,0.06
1.80,0.03,-0.02,-0.34,-0.90,-1.30,-1.23,-0.74,-0.12,0.37,0.63,0.71,0.66,0.56,0.45,0.34,0.25,0.19,0.13,0.09,0.07,0.05
1.60,0.03,0.05,-0.15,-0.81,-1.51,-1.59,-0.98,-0.13,0.51,0.81,0.85,0.75,0.60,0.46,0.34,0.24,0.17,0.12,0.08,0.05,0.04
1.40,0.02,0.08,0.07,-0.52,-1.71,-2.21,-1.38,-0.08,0.79,1.07,1.01,0.83,0.63,0.45,0.32,0.22,0.15,0.10,0.06,0.04,0.03
1.20,0.01,0.05,0.19,0.13,-1.56,-3.56,-2.16,0.32,1.35,1.39,1.14,0.87,0.63,0.43,0.29,0.19,0.12,0.07,0.05,0.03,0.02
1.00,0.00,0.02,0.13,0.38,0.82,1.32,1.72,1.88,1.80,1.55,1.21,0.89,0.61,0.40,0.25,0.15,0.09,0.05,0.03,0.02,0.01
0.80,0.00,0.01,0.06,0.25,0.67,1.26,1.80,2.08,2.02,1.71,1.29,0.89,0.57,0.34,0.19,0.10,0.05,0.03,0.01,0.01,0.00
0.60,0.00,0.00,0.02,0.12,0.46,1.11,1.86,2.34,2.33,1.93,1.37,0.86,0.48,0.25,0.12,0.05,0.02,0.01,0.00,0.00,0.00
0.40,0.00,0.00,0.00,0.02,0.19,0.79,1.82,2.72,2.86,2.26,1.42,0.73,0.32,0.13,0.04,0.01,0.00,0.00,0.00,0.00,0.00
0.20,0.00,0.00,0.00,0.00,0.01,0.22,1.32,3.30,4.05,2.80,1.21,0.35,0.07,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
